In [1]:
from copy import deepcopy
import pygame
import datetime
import numpy as np
import copy 
import csv
import ast
import math

white = (255, 255, 255)
black = (0, 0, 0)
grey = (127, 127, 127)

WIN_WIDTH = 800
WIN_HEIGHT = 400
FPS = 60

pygame.init()
pygame.mixer.init()

w = pygame.transform.scale(pygame.image.load('media/w.png'), (50, 50))
b = pygame.transform.scale(pygame.image.load('media/b.png'), (50, 50))
wob = pygame.transform.scale(pygame.image.load('media/wob.png'), (50, 50))
bob = pygame.transform.scale(pygame.image.load('media/bob.png'), (50, 50))
wwob = pygame.transform.scale(pygame.image.load('media/wwob.png'), (50, 50))
bbob = pygame.transform.scale(pygame.image.load('media/bbob.png'), (50, 50))

sound = pygame.mixer.Sound('media/knock.wav')
font = pygame.font.Font('consola.ttf', 30)
big_font = pygame.font.Font('consola.ttf', 50)

screen = pygame.display.set_mode((WIN_WIDTH, WIN_HEIGHT), 0, 200)
screen.fill(grey)
pygame.display.set_caption("English checkers")
icon = pygame.image.load('media/checker-board.jpg')
pygame.display.set_icon(icon)


class Tile(pygame.sprite.Sprite):
    def __init__(self, clr, x, y):
        super().__init__()
        
        self.clr = clr
        self.column = x
        self.row = y
        self.image = pygame.Surface((50, 50))
        self.rect = self.image.get_rect()
        self.rect.x = 50 * (x-1)
        self.rect.y = 350-(50 * (y-1))

        if self.row in range(1, 4) and (self.row + self.column)%2 == 0:
            self.piece = 1
        elif self.row in range(6, 9) and (self.row + self.column)%2 == 0:
            self.piece = -1
        else:
            self.piece = 0

        self.draw()
        
    def draw(self):
        if self.clr == 1:
            self.image = w
        elif self.clr == -1:
            if self.piece == 0:
                self.image = b
            elif self.piece == 1:
                self.image = wob
            elif self.piece == -1:
                self.image = bob
            elif self.piece == 2:
                self.image = wwob
            elif self.piece == -2:
                self.image = bbob
      
        screen.blit(self.image, (self.rect.x, self.rect.y))
        
        
class Button(pygame.sprite.Sprite):
    global screen_width, screen_height, screen

    def __init__(self, x, y, width, height, text_color, background_color, text, time=None):
        super().__init__()

        self.rect = pygame.Rect(x, y, width, height)
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.text = text
        self.text_color = text_color
        self.background_color = background_color
        self.time = time 
        
    @staticmethod
    def drawTextcenter(text, font, screen, x, y, color):
        textobj = font.render(text, True, color)
        textrect = textobj.get_rect(center=(int(x), int(y)))
        screen.blit(textobj, textrect)

    def draw(self):
        pygame.draw.rect(screen, self.background_color, (self.rect), 0)
        self.drawTextcenter(self.text, font, screen, self.x + self.width / 2, self.y + self.height / 2, self.text_color)
        pygame.draw.rect(screen, self.text_color, self.rect, 3)

    def clicked(self, pos, time):
        if self.rect.collidepoint(pos):
            if self.time is None:
                self.time = pygame.time.get_ticks()
                return True
            elif time - self.time > 1000:
                self.time = time
                return True
        return False

    
class Game:
    def __init__(self):
        self.tile_list = pygame.sprite.Group()
        self.pos_list = pygame.sprite.Group()
        self.menu = pygame.sprite.Group()
        
        cl = 1
        for row in range(8):
            cl *= -1
            for column in range(8):
                tile = Tile(cl, column + 1, row + 1)
                cl *= -1
                
                self.tile_list.add(tile)
                self.pos_list.add(tile)
                
        self.tile_list.draw(screen)
        self.running = True
        self.team = -1
        self.turn = 1 
        self.move = []
        self.base = None
        
        print('Turn # 1 by player black')
        
        def sigmoid(x):
            return 1 / (1 + np.exp(-x))
    
        def sigmoid_derivative(x):
            return x * (1 - x)

        data_list = []
       
        with open('dataset.csv', 'r') as file:
            rd = csv.reader(file)
            for row in rd:
                dlist = [ast.literal_eval(i) for i in row]
                data_list.append(dlist)

        l = len(data_list)
        dd =[]

        for i in range(l):
            for i2 in range (0, 8):
                for i3 in range (0,8):
                    dd.append(data_list[i][0][i2][i3])
            
        length = len(dd)
        inp = []
        
        for i in range(0, length, 64):
                posit = dd[i:i+64]
                inp.append(posit)
                            
        np_inp = np.array(inp)
        dd2 = []
        
        for i in range(l):
            dd2.append(data_list[i][0][8])

        np_outp = np.array(dd2)
        
        training_inputs = np_inp
        training_outputs = np_outp

        np.random.seed(1)

        self.synaptic_weights_1 = 2 * np.random.random((len(np_inp[0]), 4)) - 1
        self.synaptic_weights_2 = 2 * np.random.random((4, 4)) - 1
        self.synaptic_weights_3 = 2 * np.random.random((4, 1)) - 1 

        for iteration in range(10000):
            input_layer = training_inputs
            hidden_layer_1 = sigmoid(np.dot(input_layer, self.synaptic_weights_1))
            hidden_layer_2 = sigmoid(np.dot(hidden_layer_1, self.synaptic_weights_2))
            outputs = sigmoid(np.dot(hidden_layer_2, self.synaptic_weights_3))

            output_error = training_outputs - outputs
            output_adjustments = output_error * sigmoid_derivative(outputs)
            hidden_error_2 = np.dot(output_adjustments, self.synaptic_weights_3.T)
            hidden_adjustments_2 = hidden_error_2 * sigmoid_derivative(hidden_layer_2)
            hidden_error_1 = np.dot(hidden_adjustments_2, self.synaptic_weights_2.T)
            hidden_adjustments_1 = hidden_error_1 * sigmoid_derivative(hidden_layer_1)

            self.synaptic_weights_3 += np.dot(hidden_layer_2.T, output_adjustments)
            self.synaptic_weights_2 += np.dot(hidden_layer_1.T, hidden_adjustments_2)
            self.synaptic_weights_1 += np.dot(input_layer.T, hidden_adjustments_1)

        self._start_time = pygame.time.get_ticks()
        self.game_loop()

    def next_turn(self):
        self.base = None
        self.move = []
        self.turn += 1
        self.team *= -1
        
        print('Turn #', self.turn, 'by player', end=' ')
        
        if self.team == 1:
            print('white')
        else:
            print('black')

    def win(self): 
        black_stack = 0
        white_stack = 0

        for elem in self.tile_list:
            for row1 in range(1, 9):
                for column1 in range(1, 9):
                    if elem.row == row1 and elem.column == column1:
                        if elem.piece == 1 or elem.piece == 2:
                            white_stack += 1
                       
            for row2 in range(1, 9):
                for column2 in range(1, 9):
                    if elem.row == row2 and elem.column == column2:
                        if elem.piece == -1 or elem.piece == -2:
                            black_stack += 1
                     
        if white_stack == 0:
            print('Black won')
            self.congratulate('Black')
            return True
        elif black_stack == 0:
            print('White won')
            self.congratulate('White')
            return True

        return False
    
    def white_win(self): 
        black_stack = 0
                    
        for elem in self.tile_list:
            for row2 in range(1, 9):
                for column2 in range(1, 9):
                    if elem.row == row2 and elem.column == column2:
                        if elem.piece == -1 or elem.piece == -2:
                            black_stack += 1
                     
        if black_stack == 0:
            return True

        return False
    
    def black_win(self): 
        white_stack = 0

        for elem in self.tile_list:
            for row1 in range(1, 9):
                for column1 in range(1, 9):
                    if elem.row == row1 and elem.column == column1:
                        if elem.piece == 1 or elem.piece == 2:
                            white_stack += 1
                       
        if white_stack == 0:
            return True

        return False
    
    def white_checker_count(self):
        white_stack = 0

        for elem in self.tile_list:
            for row1 in range(1, 9):
                for column1 in range(1, 9):
                    if elem.row == row1 and elem.column == column1:
                        if elem.piece == 1 or elem.piece == 2:
                            white_stack += 1
                       
        return white_stack
    
    def white_one_more_move(self, pos1):
        for checker in self.tile_list:
            if checker.piece == 1 and checker.piece == self.team:
                for middle in self.tile_list:
                    if (middle.row == checker.row + 1) and (middle.column == checker.column + 1):
                        if (middle.piece < 0):
                            for f in self.tile_list:
                                if (f.row == checker.row + 2 ) and (f.column == checker.column +2):
                                    if f.piece == 0:
                                            return True   
                                    else:
                                            return False
                                        
                    elif (middle.row == checker.row + 1) and (middle.column == checker.column - 1):
                        if (middle.piece < 0):
                            for f in self.tile_list:
                                if (f.row == checker.row + 2 ) and (f.column == checker.column - 2):
                                    if f.piece == 0:
                                            return True
                                    else:
                                            return False
                                        
            elif checker.piece == 2 and checker.piece == self.team*2:
                for middle in self.tile_list:
                    if (middle.row == checker.row + 1) and (middle.column == checker.column + 1):
                        if (middle.piece < 0):
                            for f in self.tile_list:
                                if (f.row == checker.row + 2) and (f.column == checker.column +2):
                                    if f.piece == 0:
                                            return True
                                    else:
                                            return False
                        
                    elif (middle.row == checker.row +1) and (middle.column == checker.column - 1):
                        if (middle.piece < 0):
                            for f in self.tile_list:
                                if (f.row == checker.row +2 ) and (f.column == checker.column - 2):
                                    if f.piece == 0:
                                            return True
                                    else:
                                            return False

                    elif (middle.row == checker.row -1) and (middle.column == checker.column + 1):
                        if (middle.piece < 0):
                            for f in self.tile_list:
                                if (f.row == checker.row -2 ) and (f.column == checker.column + 2):
                                    if f.piece == 0:
                                            return True
                                    else:
                                            return False

                    elif (middle.row == checker.row -1) and (middle.column == checker.column - 1):
                        if (middle.piece < 0):
                            for f in self.tile_list:
                                if (f.row == checker.row -2 ) and (f.column == checker.column - 2):
                                    if f.piece == 0:
                                            return True
                                    else:
                                            return False
            
    def black_checker_count(self): 
        black_stack = 0
        
        for elem in self.tile_list:              
            for row2 in range(1, 9):
                for column2 in range(1, 9):
                    if elem.row == row2 and elem.column == column2:
                        if elem.piece == -1 or elem.piece == -2:
                            black_stack += 1
                            
        return black_stack

    def black_one_more_move(self, pos1):
        for checker in self.tile_list:
            if checker.piece == -1 and checker.piece == self.team:
                for middle in self.tile_list:
                    if (middle.row == checker.row - 1) and (middle.column == checker.column + 1):
                        if (middle.piece > 0):
                            for f in self.tile_list:
                                if (f.row == checker.row - 2 ) and (f.column == checker.column +2):
                                    if f.piece == 0:
                                            return True
                                    else:
                                            return False
                                        
                    elif (middle.row == checker.row - 1) and (middle.column == checker.column - 1):
                        if (middle.piece > 0):
                            for f in self.tile_list:
                                if (f.row == checker.row - 2) and (f.column == checker.column - 2):
                                    if f.piece == 0:
                                            return True
                                    else:
                                            return False
                       
            elif checker.piece == -2 and checker.piece == self.team*2:
                for middle in self.tile_list:
                    if (middle.row == checker.row + 1) and (middle.column == checker.column + 1):
                        if (middle.piece > 0):
                            for f in self.tile_list:
                                if (f.row == checker.row + 2) and (f.column == checker.column +2):
                                    if f.piece == 0:
                                            return True
                                    else:
                                            return False
                                        
                    elif (middle.row == checker.row +1) and (middle.column == checker.column - 1):
                        if (middle.piece > 0):
                            for f in self.tile_list:
                                if (f.row == checker.row +2 ) and (f.column == checker.column - 2):
                                    if f.piece == 0:
                                            return True
                                    else:
                                            return False

                    elif (middle.row == checker.row -1) and (middle.column == checker.column + 1):
                        if (middle.piece > 0):
                            for f in self.tile_list:
                                if (f.row == checker.row -2 ) and (f.column == checker.column + 2):
                                    if f.piece == 0:
                                            return True
                                    else:
                                            return False

                    elif (middle.row == checker.row -1) and (middle.column == checker.column - 1):
                        if (middle.piece > 0):
                            for f in self.tile_list:
                                if (f.row == checker.row -2 ) and (f.column == checker.column - 2):
                                    if f.piece == 0:
                                            return True
                                    else:
                                            return False
        
    def legal(self, pos1, pos2): 
        if pos1.piece == 0:
            return False
        
        if pos1.piece > 0:
            if pos1.piece != self.team and pos1.piece - 1 != self.team:
                return False
        
        if pos1.piece < 0:
            if pos1.piece != self.team and pos1.piece + 1 != self.team:
                return False
        
        if self.base is not None and pos1 != self.base:
            return False
        
        if pos2.piece != 0:
            return False
        
        if pos2.row == pos1.row or pos2.column == pos1.column:
            return False

        if pos1.piece == 1:
            if abs(pos1.column - pos2.column) != pos2.row - pos1.row:
                return False
            
        if pos1.piece == -1:
            if abs(pos1.column - pos2.column) != pos1.row - pos2.row:
                return False
            
        if pos1.piece == 2 or pos1.piece == -2:
            if abs(pos1.column - pos2.column) != abs(pos1.row - pos2.row):
                return False
            
        if abs(pos1.row - pos2.row) > 2 or abs(pos1.column - pos2.column) > 2:
            return False
        
        if abs(pos1.row - pos2.row) == 2 and abs(pos1.column - pos2.column) == 2:
                for middle in self.tile_list:
                    if middle.row == min(pos1.row, pos2.row) + 1 and middle.column == min(pos1.column, pos2.column) + 1:
                        if middle.piece == 0 or middle.piece == self.team or middle.piece == self.team*2: 
                            return False
                        
        list = []
        
        for checker in self.tile_list:
            if checker.piece == -1 and checker.piece == self.team:
                for middle in self.tile_list:
                    if (middle.row == checker.row - 1) and (middle.column == checker.column + 1):
                        if (middle.piece > 0):
                            for f in self.tile_list:
                                if (f.row == checker.row - 2 ) and (f.column == checker.column +2):
                                    if f.piece == 0:
                                        list.append(checker.row)
                                        list.append(checker.column)
                                        list.append(middle.row)
                                        list.append(middle.column)
                                        list.append(f.row)
                                        list.append(f.column)
                
                    if (middle.row == checker.row - 1) and (middle.column == checker.column - 1):
                        if (middle.piece > 0):
                            for f in self.tile_list:
                                if (f.row == checker.row - 2) and (f.column == checker.column - 2):
                                    if f.piece == 0:
                                        list.append(checker.row)
                                        list.append(checker.column)
                                        list.append(middle.row)
                                        list.append(middle.column)
                                        list.append(f.row)
                                        list.append(f.column)

            if checker.piece == 1 and checker.piece == self.team:
                for middle in self.tile_list:
                    if (middle.row == checker.row + 1) and (middle.column == checker.column + 1):
                        if (middle.piece < 0):
                            for f in self.tile_list:
                                if (f.row == checker.row + 2 ) and (f.column == checker.column +2):
                                    if f.piece == 0:
                                        list.append(checker.row)
                                        list.append(checker.column)
                                        list.append(middle.row)
                                        list.append(middle.column)
                                        list.append(f.row)
                                        list.append(f.column)
                                
                    if (middle.row == checker.row + 1) and (middle.column == checker.column - 1):
                        if (middle.piece < 0):
                            for f in self.tile_list:
                                if (f.row == checker.row + 2 ) and (f.column == checker.column - 2):
                                    if f.piece == 0:
                                        list.append(checker.row)
                                        list.append(checker.column)
                                        list.append(middle.row)
                                        list.append(middle.column)
                                        list.append(f.row)
                                        list.append(f.column)

            if checker.piece == -2 and checker.piece == self.team*2:
                for middle in self.tile_list:
                    if (middle.row == checker.row + 1) and (middle.column == checker.column + 1):
                        if (middle.piece > 0):
                            for f in self.tile_list:
                                if (f.row == checker.row + 2) and (f.column == checker.column +2):
                                    if f.piece == 0:
                                        list.append(checker.row)
                                        list.append(checker.column)
                                        list.append(middle.row)
                                        list.append(middle.column)
                                        list.append(f.row)
                                        list.append(f.column)
                                        
                    if (middle.row == checker.row +1) and (middle.column == checker.column - 1):
                        if (middle.piece > 0):
                            for f in self.tile_list:
                                if (f.row == checker.row +2 ) and (f.column == checker.column - 2):
                                    if f.piece == 0:
                                        list.append(checker.row)
                                        list.append(checker.column)
                                        list.append(middle.row)
                                        list.append(middle.column)
                                        list.append(f.row)
                                        list.append(f.column)

                    if (middle.row == checker.row -1) and (middle.column == checker.column + 1):
                        if (middle.piece > 0):
                            for f in self.tile_list:
                                if (f.row == checker.row -2 ) and (f.column == checker.column + 2):
                                    if f.piece == 0:
                                        list.append(checker.row)
                                        list.append(checker.column)
                                        list.append(middle.row)
                                        list.append(middle.column)
                                        list.append(f.row)
                                        list.append(f.column)

                    if (middle.row == checker.row -1) and (middle.column == checker.column - 1):
                        if (middle.piece > 0):
                            for f in self.tile_list:
                                if (f.row == checker.row -2 ) and (f.column == checker.column - 2):
                                    if f.piece == 0:
                                        list.append(checker.row)
                                        list.append(checker.column)
                                        list.append(middle.row)
                                        list.append(middle.column)
                                        list.append(f.row)
                                        list.append(f.column)
                                        
            if checker.piece == 2 and checker.piece == self.team*2:
                for middle in self.tile_list:
                    if (middle.row == checker.row + 1) and (middle.column == checker.column + 1):
                        if (middle.piece < 0):
                            for f in self.tile_list:
                                if (f.row == checker.row + 2) and (f.column == checker.column +2):
                                    if f.piece == 0:
                                        list.append(checker.row)
                                        list.append(checker.column)
                                        list.append(middle.row)
                                        list.append(middle.column)
                                        list.append(f.row)
                                        list.append(f.column)
                        
                    if (middle.row == checker.row +1) and (middle.column == checker.column - 1):
                        if (middle.piece < 0):
                            for f in self.tile_list:
                                if (f.row == checker.row +2 ) and (f.column == checker.column - 2):
                                    if f.piece == 0:
                                        list.append(checker.row)
                                        list.append(checker.column)
                                        list.append(middle.row)
                                        list.append(middle.column)
                                        list.append(f.row)
                                        list.append(f.column)

                    if (middle.row == checker.row -1) and (middle.column == checker.column + 1):
                        if (middle.piece < 0):
                            for f in self.tile_list:
                                if (f.row == checker.row -2 ) and (f.column == checker.column + 2):
                                    if f.piece == 0:
                                        list.append(checker.row)
                                        list.append(checker.column)
                                        list.append(middle.row)
                                        list.append(middle.column)
                                        list.append(f.row)
                                        list.append(f.column)

                    if (middle.row == checker.row -1) and (middle.column == checker.column - 1):
                        if (middle.piece < 0):
                            for f in self.tile_list:
                                if (f.row == checker.row -2 ) and (f.column == checker.column - 2):
                                    if f.piece == 0:
                                        list.append(checker.row)
                                        list.append(checker.column)
                                        list.append(middle.row)
                                        list.append(middle.column)
                                        list.append(f.row)
                                        list.append(f.column)

        p = len(list)
        
        if p == 6:
            if (pos1.row == list[0]) and (pos1.column == list[1]) and (pos2.row == list[4]) and (pos2.column == list[5]):
                for middle in self.tile_list:
                    if (middle.row == list[2]) and (middle.column == list[3]):
                        middle.piece =0
                        middle.draw()
                        return True
            else:
                return False
     
        elif p == 12:
            if (pos1.row == list[0]) and (pos1.column == list[1]) and (pos2.row == list[4]) and (pos2.column == list[5]):
                for middle in self.tile_list:
                    if (middle.row == list[2]) and (middle.column == list[3]):
                        middle.piece =0
                        middle.draw()
                        return True
 
            elif (pos1.row == list[6]) and (pos1.column == list[7]) and (pos2.row == list[10]) and (pos2.column == list[11]):
                for middle in self.tile_list:
                    if (middle.row == list[8]) and (middle.column == list[9]):
                        middle.piece = 0
                        middle.draw()
                        return True
            else:
                return False   
        
        if self.base is None:
            return True
        else:
            return False

    def legal2(self, pos1, pos2): 
        if pos1.piece == 0:
            return False
        
        if pos1.piece > 0:
            if pos1.piece != self.team and pos1.piece - 1 != self.team:
                return False
        
        if pos1.piece < 0:
            if pos1.piece != self.team and pos1.piece + 1 != self.team:
                return False
        
        if self.base is not None and pos1 != self.base:
            return False
        
        if pos2.piece != 0: 
            return False
        
        if pos2.row == pos1.row or pos2.column == pos1.column:
            return False

        if pos1.piece == 1:
            if abs(pos1.column - pos2.column) != pos2.row - pos1.row:
                return False
            
        if pos1.piece == -1:
            if abs(pos1.column - pos2.column) != pos1.row - pos2.row:
                return False
            
        if pos1.piece == 2 or pos1.piece == -2:
            if abs(pos1.column - pos2.column) != abs(pos1.row - pos2.row):
                return False
            
        if abs(pos1.row - pos2.row) > 2 or abs(pos1.column - pos2.column) > 2:
            return False
        
        if abs(pos1.row - pos2.row) == 2 and abs(pos1.column - pos2.column) == 2:
                for middle in self.pos_list:
                    if middle.row == min(pos1.row, pos2.row) + 1 and middle.column == min(pos1.column, pos2.column) + 1:
                        if middle.piece == 0 or middle.piece == self.team or middle.piece == self.team*2: 
                            return False
                        
        list = []
        
        for checker in self.pos_list:
            if checker.piece == -1 and checker.piece == self.team:
                for middle in self.pos_list:
                    if (middle.row == checker.row - 1) and (middle.column == checker.column + 1):
                        if (middle.piece > 0):
                            for f in self.pos_list:
                                if (f.row == checker.row - 2 ) and (f.column == checker.column +2):
                                    if f.piece == 0:
                                        list.append(checker.row)
                                        list.append(checker.column)
                                        list.append(middle.row)
                                        list.append(middle.column)
                                        list.append(f.row)
                                        list.append(f.column)
                
                    if (middle.row == checker.row - 1) and (middle.column == checker.column - 1):
                        if (middle.piece > 0):
                            for f in self.pos_list:
                                if (f.row == checker.row - 2) and (f.column == checker.column - 2):
                                    if f.piece == 0:
                                        list.append(checker.row)
                                        list.append(checker.column)
                                        list.append(middle.row)
                                        list.append(middle.column)
                                        list.append(f.row)
                                        list.append(f.column)

            if checker.piece == 1 and checker.piece == self.team:
                for middle in self.pos_list:
                    if (middle.row == checker.row + 1) and (middle.column == checker.column + 1):
                        if (middle.piece < 0):
                            for f in self.pos_list:
                                if (f.row == checker.row + 2 ) and (f.column == checker.column +2):
                                    if f.piece == 0:
                                        list.append(checker.row)
                                        list.append(checker.column)
                                        list.append(middle.row)
                                        list.append(middle.column)
                                        list.append(f.row)
                                        list.append(f.column)

                    if (middle.row == checker.row + 1) and (middle.column == checker.column - 1):
                        if (middle.piece < 0):
                            for f in self.pos_list:
                                if (f.row == checker.row + 2 ) and (f.column == checker.column - 2):
                                    if f.piece == 0:
                                        list.append(checker.row)
                                        list.append(checker.column)
                                        list.append(middle.row)
                                        list.append(middle.column)
                                        list.append(f.row)
                                        list.append(f.column)

            if checker.piece == -2 and checker.piece == self.team*2:
                for middle in self.pos_list:
                    if (middle.row == checker.row + 1) and (middle.column == checker.column + 1):
                        if (middle.piece > 0):
                            for f in self.pos_list:
                                if (f.row == checker.row + 2) and (f.column == checker.column +2):
                                    if f.piece == 0:
                                        list.append(checker.row)
                                        list.append(checker.column)
                                        list.append(middle.row)
                                        list.append(middle.column)
                                        list.append(f.row)
                                        list.append(f.column)
                                        
                    if (middle.row == checker.row +1) and (middle.column == checker.column - 1):
                        if (middle.piece > 0):
                            for f in self.pos_list:
                                if (f.row == checker.row +2 ) and (f.column == checker.column - 2):
                                    if f.piece == 0:
                                        list.append(checker.row)
                                        list.append(checker.column)
                                        list.append(middle.row)
                                        list.append(middle.column)
                                        list.append(f.row)
                                        list.append(f.column)

                    if (middle.row == checker.row -1) and (middle.column == checker.column + 1):
                        if (middle.piece > 0):
                            for f in self.pos_list:
                                if (f.row == checker.row -2 ) and (f.column == checker.column + 2):
                                    if f.piece == 0:
                                        list.append(checker.row)
                                        list.append(checker.column)
                                        list.append(middle.row)
                                        list.append(middle.column)
                                        list.append(f.row)
                                        list.append(f.column)

                    if (middle.row == checker.row -1) and (middle.column == checker.column - 1):
                        if (middle.piece > 0):
                            for f in self.pos_list:
                                if (f.row == checker.row -2 ) and (f.column == checker.column - 2):
                                    if f.piece == 0:
                                        list.append(checker.row)
                                        list.append(checker.column)
                                        list.append(middle.row)
                                        list.append(middle.column)
                                        list.append(f.row)
                                        list.append(f.column)
        
            if checker.piece == 2 and checker.piece == self.team*2:
                for middle in self.pos_list:
                    if (middle.row == checker.row + 1) and (middle.column == checker.column + 1):
                        if (middle.piece < 0):
                            for f in self.pos_list:
                                if (f.row == checker.row + 2) and (f.column == checker.column +2):
                                    if f.piece == 0:
                                        list.append(checker.row)
                                        list.append(checker.column)
                                        list.append(middle.row)
                                        list.append(middle.column)
                                        list.append(f.row)
                                        list.append(f.column)
                        
                    if (middle.row == checker.row +1) and (middle.column == checker.column - 1):
                        if (middle.piece < 0):
                            for f in self.pos_list:
                                if (f.row == checker.row +2 ) and (f.column == checker.column - 2):
                                    if f.piece == 0:
                                        list.append(checker.row)
                                        list.append(checker.column)
                                        list.append(middle.row)
                                        list.append(middle.column)
                                        list.append(f.row)
                                        list.append(f.column)

                    if (middle.row == checker.row -1) and (middle.column == checker.column + 1):
                        if (middle.piece < 0):
                            for f in self.pos_list:
                                if (f.row == checker.row -2 ) and (f.column == checker.column + 2):
                                    if f.piece == 0:
                                        list.append(checker.row)
                                        list.append(checker.column)
                                        list.append(middle.row)
                                        list.append(middle.column)
                                        list.append(f.row)
                                        list.append(f.column)

                    if (middle.row == checker.row -1) and (middle.column == checker.column - 1):
                        if (middle.piece < 0):
                            for f in self.pos_list:
                                if (f.row == checker.row -2 ) and (f.column == checker.column - 2):
                                    if f.piece == 0:
                                        list.append(checker.row)
                                        list.append(checker.column)
                                        list.append(middle.row)
                                        list.append(middle.column)
                                        list.append(f.row)
                                        list.append(f.column)
                                        
        p = len(list)
        
        if p == 6:
            if (pos1.row == list[0]) and (pos1.column == list[1]) and (pos2.row == list[4]) and (pos2.column == list[5]):
                for middle in self.pos_list:
                    if (middle.row == list[2]) and (middle.column == list[3]):
                        return True
            else:
                return False
     
        elif p == 12:
            if (pos1.row == list[0]) and (pos1.column == list[1]) and (pos2.row == list[4]) and (pos2.column == list[5]):
                for middle in self.pos_list:
                    if (middle.row == list[2]) and (middle.column == list[3]):
                        return True
 
            elif (pos1.row == list[6]) and (pos1.column == list[7]) and (pos2.row == list[10]) and (pos2.column == list[11]):
                for middle in self.pos_list:
                    if (middle.row == list[8]) and (middle.column == list[9]):
                        return True
            else:
                return False
    
        if self.base is None:
            return True
        else:
            return False
        
    def swap(self, pos1, pos2):
        pos1.piece, pos2.piece = pos2.piece, pos1.piece
        
        pos1.draw()
        pos2.draw()
        sound.play()
        
        print(chr(96+pos1.column)+str(pos1.row)+' '+chr(96+pos2.column)+str(pos2.row)) 

    def king (self, pos1, pos2):
        if pos1.piece == 1:
            if pos2.row == 8:
                pos1.piece = 2
                
        if pos1.piece == -1:
            if pos2.row == 1:
                pos1.piece = -2
              
    def game_loop(self):
        positions = []
        
        while True:
            pygame.time.delay(FPS)
                 
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    return False
                
                if event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE and self.team == -1:
                        position = self.position()
                        positions.append(position)
                    
                        if self.white_win():
                                for pos in positions:
                                    pos.append([1])
                                
                                with open('dataset.csv', 'a', newline='\n') as file:
                                    wr = csv.writer(file)
                                    for pos in positions:
                                        wr.writerow([pos])
                        
                        if self.black_win():
                                for pos in positions:
                                    pos.append([0])
                        
                                with open('dataset.csv', 'a', newline='\n') as file:
                                    wr = csv.writer(file)
                                    for pos in positions:
                                        wr.writerow([pos])
                        
                        position = self.position()
                        p_moves = self.possible_moves(position)
                        child = self.generate_child(position, p_moves)
                        
                        l = len(child)
                        dd =[]

                        for i in range(l):
                            for i2 in range (0, 8):
                                for i3 in range (0,8):
                                    dd.append(child[i][i2][i3])
        
                        length = len(dd)
                        inp2 = []
                
                        for i in range(0, length, 64):
                            posit = dd[i:i+64]
                            inp2.append(posit)
                            
                        w = []
                        
                        for ch in inp2:
                            input_layer2 = np.array(ch)
                            hidden_layer_11 = self.sigmoid(np.dot(input_layer2, self.synaptic_weights_1))
                            hidden_layer_22 = self.sigmoid(np.dot(hidden_layer_11, self.synaptic_weights_2))
                            output2 = self.sigmoid(np.dot(hidden_layer_22, self.synaptic_weights_3))
                            value = output2[0]
                            ww = [value, ch]
                            w.append(ww)
                            
                        min_o = float('+inf')
                        
                        for one2 in w:
                            o2 = one2[0]
                            min_o = min(o2, min_o)
                            
                        rezul = []
                        
                        for i in w:
                            if i[0] == min_o:
                                rezul.append(i[1])
                                
                        rezul2 = rezul[0]
                        eight_checkers = []
                        
                        for i in range(0, 64, 8):
                            eig = rezul2[i:i+8]
                            eight_checkers.append(eig)
                            
                        counter = 0
                        coun = 0
                        
                        for ch in child:
                            if ch == eight_checkers:
                                coun = counter
                            counter = counter + 1
                        
                        move = p_moves[coun]
                        move2 = []
                        
                        for i in range (0, 4, 2):
                            p = move[i:i+2]
                            move2.append(p)
                            
                        xx = 0
                        yy = 0
                        
                        for move in move2:
                            xx = 50*(move[1]-0.5)
                            yy = 50*(abs(move[0]-8.5))

                            tuplee = (xx, yy)
                            
                            for i in self.tile_list:
                                if i.rect.collidepoint(tuplee):
                                    if self.running:
                                        self.move.append(i)
                                        if len(self.move) == 2:
                                            before1 = self.white_checker_count()
                                            before2 = self.black_checker_count() 
                                            if self.legal(self.move[0], self.move[1]):
                                                self.king(self.move[0], self.move[1])
                                                self.swap(self.move[0], self.move[1])
                                                self.running = not self.win()
                                                
                                                after1 = self.white_checker_count()
                                                after2 = self.black_checker_count()
                                                
                                                self.base = self.move[1]
                                                self.move = [self.base]
                                                if self.running:
                                                    if before1 != after1:
                                                        if self.black_one_more_move(self.base):
                                                            print('one more move - black')
                                                        else:
                                                            self.next_turn()
                                                            
                                                    elif before2 != after2:
                                                        if self.white_one_more_move(self.base):
                                                            print('one more move - white')
                                                        else:
                                                            self.next_turn()
                                                    else:
                                                            self.next_turn()  
                                            else:
                                                if self.base is None:
                                                    self.move.remove(self.move[0])
                                                else:
                                                    self.move.remove(self.move[1])
                                                
                if event.type == pygame.KEYDOWN and event.key == pygame.K_RETURN and self.team == 1:
                        position = self.position()
                        positions.append(position)
                        
                        if self.white_win():
                            for pos in positions:
                                pos.append([1])
                                
                            with open('dataset.csv', 'a', newline='\n') as file:
                                wr = csv.writer(file)
                                for pos in positions:
                                        wr.writerow([pos])
                                        
                        if self.black_win():
                            for pos in positions:
                                    pos.append([0])
                                   
                            with open('dataset.csv', 'a', newline='\n') as file:
                                wr = csv.writer(file)
                                for pos in positions:
                                        wr.writerow([pos])
                                        
                        position = self.position()
                        p_moves = self.possible_moves(position)
                        child = self.generate_child(position, p_moves)
        
                        d = []
                        
                        for ch in child:
                            value = self.alpha_beta(ch, 6, -math.inf, math.inf, True)
                            dd = [value, ch]
                            d.append(dd)
                            
                        max_o = float('-inf')
                        
                        for one in d:
                            o = one[0]
                            max_o=max(o, max_o)
                            
                        rez = []
                        
                        for i in d:
                            if i[0] == max_o:
                                rez.append(i[1])
                                
                        counter = 0
                        coun = 0
                        
                        for ch in child:
                            if ch == rez[0]:
                                coun = counter
                            counter = counter + 1
                        
                        move = p_moves[coun]
                        move2 = []
                        
                        for i in range(0, 4, 2):
                            p = move[i:i+2]
                            move2.append(p)
                            
                        value2 = self.alpha_beta(position, 1, -math.inf, math.inf, True)
                        
                        xx = 0
                        yy = 0
                        
                        for move in move2:
                            xx = 50*(move[1]-0.5)
                            yy = 50*(abs(move[0]-8.5))
                            
                            tuplee = (xx, yy)
                            for i in self.tile_list:
                                if i.rect.collidepoint(tuplee):
                                    if self.running:
                                        self.move.append(i)
                                        if len(self.move) == 2:
                                            before1 = self.white_checker_count()
                                            before2 = self.black_checker_count() 
                                            if self.legal(self.move[0], self.move[1]):
                                                self.king(self.move[0], self.move[1])
                                                self.swap(self.move[0], self.move[1])
                                                self.running = not self.win()
                                                
                                                after1 = self.white_checker_count()
                                                after2 = self.black_checker_count()
                                                
                                                self.base = self.move[1]
                                                self.move = [self.base]
                                                if self.running:
                                                    if before1 != after1:
                                                        if self.black_one_more_move(self.base):
                                                            print("one more move - black")
                                                        else:
                                                            self.next_turn()
                                                            
                                                    elif before2 != after2:
                                                        if self.white_one_more_move(self.base):
                                                            print("one more move - white")
                                                        else:
                                                            self.next_turn()
                                                    else:
                                                            self.next_turn()
                                                            
                                            else:
                                                if self.base is None:
                                                    self.move.remove(self.move[0])
                                                else:
                                                    self.move.remove(self.move[1])
                                                
                if event.type == pygame.MOUSEBUTTONDOWN and event.button == 3:
                    self.next_turn()
                  
                if event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                        pos = pygame.mouse.get_pos()
                        frame = pygame.time.get_ticks()
                        for i in self.tile_list:
                            if i.rect.collidepoint(pos):
                                if self.running:
                                    self.move.append(i)
                                    if len(self.move) == 2:
                                            before1 = self.white_checker_count()
                                            before2 = self.black_checker_count() 
                                            if self.legal(self.move[0], self.move[1]):
                                                position = self.position()
                                                positions.append(position)
                                                
                                                self.king(self.move[0], self.move[1])
                                                self.swap(self.move[0], self.move[1])
                                                self.running = not self.win()
                                                
                                                after1 = self.white_checker_count()
                                                after2 = self.black_checker_count()
                                                
                                                self.base = self.move[1]
                                                self.move = [self.base]
                                                if self.running:
                                                    if before1 != after1:
                                                        if self.black_one_more_move(self.base):
                                                            print('one more move - black')
                                                        else:
                                                            self.next_turn()
                                                    elif before2 != after2:
                                                        if self.white_one_more_move(self.base):
                                                            print('one more move -white')
                                                        else:
                                                            self.next_turn()
                                                    else:
                                                            self.next_turn()

                                            else:
                                                if self.base is None:
                                                    self.move.remove(self.move[0])
                                                else:
                                                    self.move.remove(self.move[1]) 
                                                    
                if event.type == pygame.KEYDOWN and event.key == pygame.K_TAB:
                    data_list2 = []
                    
                    with open('dataset.csv', 'r') as file:
                        rd = csv.reader(file)
                        for row in rd:
                            dlist = [ast.literal_eval(i) for i in row]
                            data_list2.append(dlist)
                            
                    l = len(data_list2)
                    dd22 = []
                    dd222 = []
                    
                    for i in range(l):                
                        m = [data_list2[i][0][0], data_list2[i][0][1], data_list2[i][0][2], data_list2[i][0][3], data_list2[i][0][4], data_list2[i][0][5], data_list2[i][0][6], data_list2[i][0][7]]         
                        dd22.append(m)
                        
                    ll = len(dd22)
                    
                    for i in range(ll):
                        val = self.alpha_beta(dd22[i], 7, -math.inf, math.inf, False)
                        if val >= 0:
                            val3 = 1
                            
                        if val < 0:
                            val3 = 0
                            
                        dd22[i].append([val3])
                        
                    with open('dataset_alfa_beta.csv', 'a', newline='\n') as file:
                                wr = csv.writer(file)
                                for pos in dd22:
                                    wr.writerow([pos])
                    
            self.tile_list.update()
            pygame.display.update()
            self.widget()
                                          
    def widget(self):
        first = 'Time: ' + str(datetime.timedelta(seconds=(pygame.time.get_ticks() - self._start_time) / 1000))[:7]
        second = 'Player: ' + {1: 'White', -1: 'Black'}[self.team]
        third = 'Turn: ' + str(self.turn)
        
        background = pygame.Surface((400, 200))
        background.fill(grey)
        screen.blit(background, (800, 0))

        widget_surface = pygame.Surface((0, 0))
        widget_surface.fill(grey)
        i = 0
        
        for el in [first, second, third]:
            widget = font.render(el, True, white, grey)
            widgetRect = widget.get_rect()
            widgetRect.center = (600, 50*(1+i))
            screen.blit(widget, widgetRect)
            i += 1 
            
    def position(self):
        posit = [[], [], [], [], [], [], [], [], []] 
        for row in range(0, 9):
                for column in range(0, 9):
                    for tile in self.tile_list:
                        if (tile.row == row and tile.column == column):
                            if tile.piece == 1:
                                posit[abs(row-9)].append(1)
                            elif tile.piece == 2:
                                posit[abs(row-9)].append(2)
                            elif tile.piece == -1:
                                posit[abs(row-9)].append(-1)
                            elif tile.piece == -2:
                                posit[abs(row-9)].append(-2)
                            else:
                                posit[abs(row-9)].append(0)
                    
        posit.remove([])
        return posit

    def possible_moves(self, position):
        
        p_moves = []
        
       
        for row in range(1, 9):
            for column in range(1, 9):
                for pos in self.pos_list:
                    if (pos.row == row and pos.column == column):
                        if position[abs(row-8)][column-1] == 1:
                            pos.piece = 1
                        elif position[abs(row-8)][column-1] == -1:
                            pos.piece = -1
                        elif position[abs(row-8)][column-1] == 2:
                            pos.piece = 2
                        elif position[abs(row-8)][column-1] == -2:
                            pos.piece = -2
                        elif position[abs(row-8)][column-1] == 0:
                            pos.piece = 0
                        
        for pos1 in self.pos_list:
            if pos1.piece == self.team or pos1.piece == self.team*2:
                for pos2 in self.pos_list:
                    
                        if self.legal2(pos1, pos2) and (pos1.piece == self.team or pos1.piece == self.team*2) and pos2.piece == 0:
                            move = [pos1.row, pos1.column, pos2.row, pos2.column]
                            p_moves.append(move)
        
        
        l = len(p_moves)
        
        
        return p_moves
    
    def generate_child(self, posit, p_moves):
        child = []
        pm = len(p_moves) 
            
        for t in range (0, pm): 
            t0 = p_moves[t][0]
            t1 = p_moves[t][1]
            t2 = p_moves[t][2]
            t3 = p_moves[t][3]

            s = posit[abs(t0-8)][abs(t1-1)]
            f = posit[abs(t2-8)][abs(t3-1)]
            
            pos_copy = copy.deepcopy(posit)
            pos_copy[abs(t0-8)][abs(t1-1)] = f
            pos_copy[abs(t2-8)][abs(t3-1)] = s
            
            child.append(pos_copy)
            
        pp = len(child)
        return child
            
    def heur_func(self, position):
        h_value = 0
        
        index = [[1.0, 1.1, 1.2, 1.3, 1.3, 1.2, 1.1, 1.0],
                [1.0, 1.1, 1.2, 1.3, 1.3, 1.2, 1.1, 1.0],
                [1.0, 1.1, 1.2, 1.3, 1.3, 1.2, 1.1, 1.0],
                [1.0, 1.1, 1.2, 1.3, 1.3, 1.2, 1.1, 1.0],
                [1.0, 1.1, 1.2, 1.3, 1.3, 1.2, 1.1, 1.0],
                [1.0, 1.1, 1.2, 1.3, 1.3, 1.2, 1.1, 1.0],
                [1.0, 1.1, 1.2, 1.3, 1.3, 1.2, 1.1, 1.0],
                [1.0, 1.1, 1.2, 1.3, 1.3, 1.2, 1.1, 1.0]]
                 
        h = [[0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0]]
        
        for row in range(0, 8):
            for col in range(0, 8):
                    h[row][col] = position[row][col] * index[row][col]
                    
        summ = 0
        
        for row in range(0, 8):
            for col in range (0, 8):
                summ += h[row][col]
                
        h_value += summ
        return h_value
        
    def alpha_beta(self, position, depth, alpha, beta, maximizingPlayer):
        if depth == 0 or self.win():
            return self.heur_func(position)
 
        if maximizingPlayer:
            maxEval = float('-inf')
            
            p_moves = self.possible_moves(position)
            child = self.generate_child(position, p_moves)
            
            for ch in child:
                ev = self.alpha_beta(ch, depth - 1, alpha, beta, False)
                
                maxEval = max(maxEval, ev)
                alpha = max(alpha, ev)
                
                if beta <= alpha:
                    break
                    
            return maxEval
        
        else:
            minEval = float('inf')
            
            p_moves = self.possible_moves(position)
            child = self.generate_child(position, p_moves)
            
            for ch in child:
                ev = self.alpha_beta(ch, depth - 1, alpha, beta, True)
                
                minEval = min(minEval, ev)
                beta = min(beta, ev)
                
                if beta <= alpha:
                    break
                    
            return minEval
           
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def sigmoid_derivative(self, x):
        return x * (1 - x)
        
    @staticmethod
    def print_doc(doc):
        background = pygame.Surface((800, 1200))
        background.fill(grey)
        screen.blit(background, (0, 0))
        i = 0
        
    @staticmethod
    def congratulate(clr):
        background = pygame.Surface((800, 1200))
        background.fill(grey)
        screen.blit(background, (0, 0))
        sentence = big_font.render(clr + ' Wins!', True, white, grey)
        sentenceRect = sentence.get_rect()
        sentenceRect.center = (400, 400)
        screen.blit(sentence, sentenceRect)

Game()

pygame 2.3.0 (SDL 2.24.2, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
pos_list
<class 'pygame.sprite.Group'>
Turn # 1 by player black
